In [1]:
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Table, Column, Integer, String, Boolean, DateTime, Date, MetaData, ForeignKey
from sqlalchemy.engine.url import URL
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool
import csv
local_name = 'sqlite:///ods-slack.db'
engine = create_engine(local_name, echo=True)


In [2]:
users = pd.read_sql_query('select * from imported_user_data', engine)
messages = pd.read_sql_query('select * from imported_messages', engine)


2018-03-18 11:54:30,506 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-18 11:54:30,509 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 11:54:30,511 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-18 11:54:30,515 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 11:54:30,517 INFO sqlalchemy.engine.base.Engine select * from imported_user_data
2018-03-18 11:54:30,519 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 11:54:30,712 INFO sqlalchemy.engine.base.Engine select * from imported_messages
2018-03-18 11:54:30,713 INFO sqlalchemy.engine.base.Engine ()


In [4]:
print(users.shape)
print(messages.shape)

(11159, 34)
(751861, 12)


In [7]:
users.head()

,id,name,deleted,tz,tz_label,tz_offset,is_admin,is_owner,is_primary_owner,is_restricted,...,avatar_hash,first_name,last_name,image_24,image_32,image_48,image_72,image_192,image_512,team
0,U4EMUGFNG,0,0,Asia/Jerusalem,Israel Standard Time,7200,0,0,0,0,...,g204d33bf41b,0,,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,T040HKJE3
1,U7SE7ANCW,00serd00,0,Asia/Vladivostok,Vladivostok Time,36000,0,0,0,0,...,g428cf084b50,,,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,T040HKJE3
2,U6YSPBJK0,012345vov,0,Asia/Kuwait,Arabia Standard Time,10800,0,0,0,0,...,gecadf1ec70f,012345vov,,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,T040HKJE3
3,U8YE1BVPS,04ok11,0,Europe/Helsinki,Eastern European Time,7200,0,0,0,0,...,gc1c71b3389d,,,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,T040HKJE3
4,U6U25UP96,0xdev,0,Europe/Minsk,Further-eastern European Time,10800,0,0,0,0,...,83d3b0c58991,Piashchynski,Valery,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,T040HKJE3


In [8]:
messages.head()

,channel,ts,type,text,user,thread_ts,parent_user_id,subtype,reply_count,unread_count,bot_id,username
0,bots,1502440131.035604,message,<@U1Z2QA4EM> как избавиться от рекурсии?,U1UMQM200,,,,,,,
1,bots,1502440131.035853,message,<@U1UMQM200>: избавиться от искушения - это ка...,U1Z2QA4EM,,,,,,,
2,bots,1476874935.003142,message,<@U1Z2QA4EM> в психотерапию умеешь?,U09JEC7V0,,,,,,,
3,bots,1476874936.003143,message,<@U09JEC7V0>: ох уж этот реверс в аметисты сос...,U1Z2QA4EM,,,,,,,
4,bots,1476874950.003144,message,<@U1Z2QA4EM> может ты у мамки психолог?,U065VP6F7,,,,,,,


In [180]:
df = pd.read_sql_query(
    'select usr.id, usr.name, msg.channel, msg.ts, msg.text \
    from imported_messages as msg inner join imported_user_data as usr on msg.user = usr.id', engine)

2018-03-18 14:16:59,950 INFO sqlalchemy.engine.base.Engine select usr.id, usr.name, msg.channel, msg.ts, msg.text     from imported_messages as msg inner join imported_user_data as usr on msg.user = usr.id


INFO:sqlalchemy.engine.base.Engine:select usr.id, usr.name, msg.channel, msg.ts, msg.text     from imported_messages as msg inner join imported_user_data as usr on msg.user = usr.id


2018-03-18 14:16:59,954 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [181]:
print(df.shape)
df.head()


(740471, 5)


,id,name,channel,ts,text
0,U1UMQM200,iggisv9t,bots,1502440131.035604,<@U1Z2QA4EM> как избавиться от рекурсии?
1,U1Z2QA4EM,markov-bot,bots,1502440131.035853,<@U1UMQM200>: избавиться от искушения - это ка...
2,U09JEC7V0,ssviridov,bots,1476874935.003142,<@U1Z2QA4EM> в психотерапию умеешь?
3,U1Z2QA4EM,markov-bot,bots,1476874936.003143,<@U09JEC7V0>: ох уж этот реверс в аметисты сос...
4,U065VP6F7,libfun,bots,1476874950.003144,<@U1Z2QA4EM> может ты у мамки психолог?


In [182]:
import re
import editdistance

def get_levenstein(df, th):
    df_match = df[df.apply(lambda row: (editdistance.eval(row['name'], row['mention']) <= th and 
                           (len(row['name']) > th  * 2 + 2 or len(row['mention']) > th * 2 + 2)), axis=1)]
#     print(df_match.shape)
#     print(df_match[df_match['name'] != df_match['mention']][['name', 'mention']].head())
    return df_match[['id', 'name', 'mention']]
#     return df_match

In [183]:
github_regex = '<http[^>]*github.com/([^>/]+)'

df_github = df[df.text.str.contains(github_regex)]
df_github['mention'] = ''
for index, row in df_github.iterrows():
    df_github.loc[index]['name'] = df_github.loc[index]['name'].lower()
    m = re.search(github_regex, row['text'])
    if m:
        df_github.loc[index]['mention'] = m.group(1).lower()
        

# df_github_match = df_github[df_github.apply(lambda row: row['name'] == row['mention'], axis=1)]
# df_github_match.shape
github_match = pd.DataFrame(columns=['id', 'name', 'mention'])

for th in range(6):
    github_match = github_match.append(get_levenstein(df_github, th))
    
githubio_regex = '/(\w+).github.io'

df_githubio = df[df.text.str.contains(githubio_regex)]
df_githubio['mention'] = ''
for index, row in df_githubio.iterrows():
    df_githubio.loc[index]['name'] = df_githubio.loc[index]['name'].lower()
    m = re.search(githubio_regex, row['text'])
    if m:
        df_githubio.loc[index]['mention'] = m.group(1).lower()

for th in range(6):
    github_match = github_match.append(get_levenstein(df_githubio, th))

github_match = github_match.drop_duplicates()

print(github_match.shape)
github_match[github_match['name'] != github_match['mention']]

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:21: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(119, 3)


,id,name,mention
54493,U054CQTS7,1aguschin,aguschin
107775,U4PKEBC03,kit_scribe,kit-scribe
108265,U6MNDLDT7,philip-b,philip-bl
142333,U0D6W1WTW,alex_sosn,alexsosn
145337,U04DXFZ2G,fedor.chervinskii,fedor-chervinskii
161409,U1ZCENED7,romul,lromul
468091,U6L7YLWHY,shkarupa.alex,shkarupa-alex
636832,U4RFX4PJT,nerses_bagiyan,nersesbagiyan
636862,U04725QK7,gleb_filatov,glebfilatov
636894,U2ZJ517B8,sharlottik,sharlotik


In [184]:
kaggle_regex = '<http[^>]*kaggle.com/([^>/]+)'

df_kaggle = df[df.text.str.contains(kaggle_regex)]
df_kaggle['mention'] = ''
for index, row in df_kaggle.iterrows():
    df_kaggle.loc[index]['name'] = df_kaggle.loc[index]['name'].lower()
    m = re.search(kaggle_regex, row['text'])
    if m:
        df_kaggle.loc[index]['mention'] = m.group(1).lower()
        

# df_kaggle_match = df_kaggle[df_kaggle.apply(lambda row: row['name'] == row['mention'], axis=1)]
# df_kaggle_match.shape
kaggle_match = pd.DataFrame(columns=['id', 'name', 'mention'])

for th in range(6):
    kaggle_match = kaggle_match.append(get_levenstein(df_kaggle, th))

kaggle_match = kaggle_match.drop_duplicates()

print(kaggle_match.shape)
kaggle_match[kaggle_match['name'] != kaggle_match['mention']]

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(94, 3)


,id,name,mention
50333,U4FCBEVAS,inga.kaydanova,ingakaydanova
50335,U4XSYA99C,andrey.,andreyg
50353,U2Z07JW4T,alex.radionov,alexradionov
50372,U2T2WU8L9,fartuk,fartuk1
50373,U4HS6PVBR,alelu,alexlu
50377,U4X5SNMK2,vfdev,vfdev5
54783,U0H7VBQQ1,alex.ozerin,alexozerin
58684,U1BKBLY4X,antter7,antter
61554,U86DVN6G1,pavelosta,pavelost
243970,U436BP343,waniz,wanizz


In [185]:
linkedin_regex = '<http[^>]*linkedin.com/in/([^>/]+)'

df_linkedin = df[df.text.str.contains(linkedin_regex)]
df_linkedin['mention'] = ''
for index, row in df_linkedin.iterrows():
    df_linkedin.loc[index]['name'] = df_linkedin.loc[index]['name'].lower()
    m = re.search(linkedin_regex, row['text'])
    if m:
        df_linkedin.loc[index]['mention'] = m.group(1).lower()
        

# df_linkedin_match = df_linkedin[df_linkedin.apply(lambda row: row['name'] == row['mention'], axis=1)]
# df_linkedin_match.shape
linkedin_match = pd.DataFrame(columns=['id', 'name', 'mention'])

for th in range(6):
    linkedin_match = linkedin_match.append(get_levenstein(df_linkedin, th))

linkedin_match = linkedin_match.drop_duplicates()

print(linkedin_match.shape)
linkedin_match[linkedin_match['name'] != linkedin_match['mention']]

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


(30, 3)


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,name,mention
244076,U4GD9N3GB,valentin.mikhaylenko,valentinmikhaylenko
244081,U1NLYUY5N,leonid.danilchenko,leonid-danilchenko
253293,U049HDR2Z,egor.panfilov,egorpanfilov
253388,U0ZHCGY5T,egor_labintcev,egorlabintcev
228321,U30Q72KLJ,tchernov,timchernov
253368,U30A6KZLH,atselikov,alextselikov
548810,U3UB29PTJ,iryndin,ivanryndin
286736,U364MEXR9,a.kumpan,antonkumpan


In [186]:
best = github_match.merge(kaggle_match.merge(linkedin_match, how='inner', on=['name']), how='inner', on=['name'])
print(best)
best = github_match.merge(kaggle_match, how='inner', on=['name'])
print(best)

          id         name      mention       id_x    mention_x       id_y  \
0  U04CH4QBD  ermakovpetr  ermakovpetr  U04CH4QBD  ermakovpetr  U04CH4QBD   
1  U2LGQMC4D     venheads     venheads  U2LGQMC4D     venheads  U2LGQMC4D   

     mention_y  
0  ermakovpetr  
1     venheads  
         id_x           name      mention_x       id_y      mention_y
0   U07V1URT9          n01z3          n01z3  U07V1URT9        drn01z3
1   U0FEJNBGQ         mabrek         mabrek  U0FEJNBGQ         mabrek
2   U0JHK9001        rushter        rushter  U0JHK9001        rushter
3   U3PLVL3RD         jgc128         jgc128  U3PLVL3RD         jgc128
4   U0ZJV6E5Q      movchan74      movchan74  U0ZJV6E5Q        movchan
5   U0FNDT5NU        mike0sv        mike0sv  U0FNDT5NU        mike0sv
6   U1G303UTW     arsenyinfo     arsenyinfo  U1G303UTW     arsenyinfo
7   U04CH4QBD    ermakovpetr    ermakovpetr  U04CH4QBD    ermakovpetr
8   U6QD48HHS         xmvlad         xmvlad  U6QD48HHS         xmvlad
9   U43FTJQ2V    

In [193]:
total = pd.DataFrame(columns=['id', 'name', 'key', 'value'])

github_match['key'] = 'github'
kaggle_match['key'] = 'kaggle'
linkedin_match['key'] = 'linkedin'

github_match = github_match.rename(columns={"mention": "value"})
kaggle_match = kaggle_match.rename(columns={"mention": "value"})
linkedin_match = linkedin_match.rename(columns={"mention": "value"})

print(kaggle_match.head())
total = github_match.merge(kaggle_match.merge(linkedin_match, how='outer', on=['id', 'name', 'key', 'value']), how='outer', on=['id', 'name', 'key', 'value'])
print(total.shape)
print(total.head())
total[['id', 'key', 'value']].to_csv('user_profiles.csv', index=False)

              id         name        value     key
45085  U43FTJQ2V      zfturbo      zfturbo  kaggle
46996  U6GTG35V0  andrew32bit  andrew32bit  kaggle
50331  U1V57JCUF       tenich       tenich  kaggle
50334  U4BBNMQ03      justart      justart  kaggle
50341  U4D7EK3N3     funkindy     funkindy  kaggle
(243, 4)
          id     name    value     key
0  U07V1URT9    n01z3    n01z3  github
1  U0FEJNBGQ   mabrek   mabrek  github
2  U0JHK9001  rushter  rushter  github
3  U3PLVL3RD   jgc128   jgc128  github
4  U4P3L4R19    sh1ng    sh1ng  github


In [ ]:
total

In [ ]:
githubio_regex = '<.*/(\w+).github.io'

df_githubio = df[df.text.str.contains(githubio_regex)]
df_githubio
df_githubio['mention'] = ''
for index, row in df_githubio.iterrows():
    df_githubio.loc[index]['name'] = df_githubio.loc[index]['name'].lower()
    m = re.search(githubio_regex, row['text'])
    if m:
        df_githubio.loc[index]['mention'] = m.group(1).lower()
        

df_githubio_match = df_githubio[df_githubio.apply(lambda row: editdistance.eval(row['name'], row['mention']) <= 2, axis=1)]
print(df_githubio_match.shape)
df_githubio_match

In [76]:
df_github.head()

,user,name,channel,ts,text,github_mention
13557,U14BPHDK6,katya,bots,1480492906.015643,this <http://github.com/open-data-science/deep...,open-data-science
18397,U0FNDT5NU,mike0sv,bots,1477509715.004720,тут валяется код времен датафеста <https://git...,open-data-science
31050,U36KSMGDN,justm,bots,1488810157.020224,"я делал так\nПС. говно код, надо рефакторить <...",justm57
34907,U1B2L9G58,vykhand,bots,1488132700.019998,<https://github.com/Azure/bot-education>,azure
42291,U30A6KZLH,atselikov,kaggle_crackers,1499240982.841129,кароче по теме:\n<https://github.com/diefimov/...,diefimov


In [81]:
df_github[df_github['name'] == df_github['github_mention']]

,user,name,channel,ts,text,github_mention
43705,U07V1URT9,n01z3,kaggle_crackers,1499473725.992367,<https://github.com/N01Z3/iNaturalist>\nПока ж...,n01z3
43796,U0FEJNBGQ,mabrek,kaggle_crackers,1452942819.000158,написал подробный пост про Rossman <http://mab...,mabrek
44335,U0JHK9001,rushter,kaggle_crackers,1463239889.001124,<https://github.com/rushter/heamy>\nвыложил мо...,rushter
46351,U0JHK9001,rushter,kaggle_crackers,1500847275.804070,можешь сюда запушить <https://github.com/rusht...,rushter
46364,U3PLVL3RD,jgc128,kaggle_crackers,1500849054.940928,<https://gist.github.com/jgc128/6b3c08e093251b...,jgc128
47497,U4P3L4R19,sh1ng,kaggle_crackers,1493382637.370429,<https://github.com/sh1ng/arboretum>,sh1ng
47499,U4P3L4R19,sh1ng,kaggle_crackers,1493382788.406157,<https://github.com/sh1ng/arboretum_benchmark>,sh1ng
48540,U0ZJV6E5Q,movchan74,kaggle_crackers,1498034975.155342,А я поучаствовал. Решил попробовать применить ...,movchan74
49272,U07V1URT9,n01z3,kaggle_crackers,1496419835.594852,"Кому интересно, выложил свой код с yt8m\n<http...",n01z3
49699,U0JHK9001,rushter,kaggle_crackers,1462906553.001041,"я тут решил выложить модуль, который использую...",rushter


In [69]:
dir(str)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'zfill']

In [24]:
df[df.text.str.contains('github')].shape

(5864, 5)

In [35]:
df.to_csv('imported_messages.csv', encoding='utf-8', index=False, escapechar='\\')

In [ ]:
df

In [9]:
df = pd.read_sql_query('select * from imported_reactions_count', engine)

2018-03-17 21:04:26,405 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-17 21:04:26,409 INFO sqlalchemy.engine.base.Engine ()
2018-03-17 21:04:26,411 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-17 21:04:26,412 INFO sqlalchemy.engine.base.Engine ()
2018-03-17 21:04:26,413 INFO sqlalchemy.engine.base.Engine select * from imported_reactions_count
2018-03-17 21:04:26,414 INFO sqlalchemy.engine.base.Engine ()


In [10]:
df.shape

(169882, 4)

In [11]:
df.to_csv('imported_reactions_count.csv', encoding='utf-8', index=False, escapechar='\\')

In [12]:
df

,channel,message_ts,name,count
0,bots,1502440131.035853,coolstorybob,2
1,bots,1476875014.003147,joy,3
2,bots,1476879160.003151,motokozak,3
3,bots,1476891493.003183,putin,2
4,bots,1476893452.003206,joy,1
5,bots,1476893733.003214,java,2
6,bots,1476901498.003237,cat2,1
7,bots,1476903065.003239,but_why,1
8,bots,1476903281.003241,levenchuk,1
9,bots,1476904937.003243,putin,1
